In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\chand\anaconda3\envs\agenticaienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings.embed_query("hello AI")

[-0.03338824212551117,
 0.03453981503844261,
 0.05947450175881386,
 0.059286098927259445,
 -0.06353531032800674,
 -0.06819586455821991,
 0.08823321014642715,
 0.0344407856464386,
 -0.03278516232967377,
 -0.01581495814025402,
 0.020981721580028534,
 -0.018340323120355606,
 -0.03983224928379059,
 -0.0804707333445549,
 -0.014469229616224766,
 0.03326485678553581,
 0.014259278774261475,
 -0.03404996171593666,
 -0.142915740609169,
 -0.023083431646227837,
 -0.021380223333835602,
 0.002633583964779973,
 -0.04729269817471504,
 -0.010752756148576736,
 -0.06866802275180817,
 0.031124936416745186,
 0.07594586908817291,
 0.0011282607447355986,
 0.011632048524916172,
 -0.036039240658283234,
 0.04483754187822342,
 0.018390731886029243,
 0.12672801315784454,
 -0.0013597395736724138,
 0.00820669624954462,
 0.06909967958927155,
 -0.08076362311840057,
 -0.05841310694813728,
 0.0537545382976532,
 0.02622750587761402,
 -0.0068285781890153885,
 -0.05635844171047211,
 0.0032929808367043734,
 -0.072501882910

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [8]:
my_query="Narendra modi is prime minister of india?"

In [9]:
document_embedding=embeddings.embed_documents(documents)

In [10]:
document_embedding

[[0.11998702585697174,
  -0.02130262926220894,
  -0.042880889028310776,
  0.06645582616329193,
  -0.0643523707985878,
  -0.04424867033958435,
  0.0224084984511137,
  -0.04987304285168648,
  -0.02343769185245037,
  -0.03397204354405403,
  -0.014047956094145775,
  -0.06065933406352997,
  -0.003906781319528818,
  -0.017782054841518402,
  -0.047971002757549286,
  -0.06668156385421753,
  0.004103164188563824,
  -0.013092752546072006,
  0.0443977490067482,
  0.022350700572133064,
  0.009459548629820347,
  -0.020564574748277664,
  -0.00033560910378582776,
  -0.005685771815478802,
  0.05558698996901512,
  0.025123195722699165,
  -0.0028170919977128506,
  0.008758911862969398,
  0.003255274845287204,
  -0.015963375568389893,
  0.014263669960200787,
  -0.11220848560333252,
  0.08968566358089447,
  -0.03108374960720539,
  -0.02422380819916725,
  0.006152117159217596,
  0.08058716356754303,
  0.0182499997317791,
  0.05568310618400574,
  0.016702676191926003,
  0.015895962715148926,
  0.00034109121

In [11]:
query_embedding=embeddings.embed_query(my_query)

In [12]:
len(query_embedding)

384

In [13]:
cosine_similarity([query_embedding],document_embedding)

array([[0.11756667, 0.34324566, 0.81413235]])

In [14]:
from sklearn.metrics.pairwise import euclidean_distances

In [15]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32848281, 1.14608409, 0.60970098]])

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [18]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [19]:
index=faiss.IndexFlatL2(384)


In [20]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000019FE677A070> >

In [21]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [22]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['97026997-1bf2-4e42-b373-8d3fd7e8d7e8',
 '63214224-7d22-428a-ac51-bc199fee3bb9',
 '4499336b-c726-4d05-8e87-a9ac58025991']

In [23]:
vector_store.index_to_docstore_id

{0: '97026997-1bf2-4e42-b373-8d3fd7e8d7e8',
 1: '63214224-7d22-428a-ac51-bc199fee3bb9',
 2: '4499336b-c726-4d05-8e87-a9ac58025991'}

In [24]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [25]:
results

[Document(id='63214224-7d22-428a-ac51-bc199fee3bb9', metadata={}, page_content='AI is powerful'),
 Document(id='97026997-1bf2-4e42-b373-8d3fd7e8d7e8', metadata={}, page_content='AI is future'),
 Document(id='4499336b-c726-4d05-8e87-a9ac58025991', metadata={}, page_content='Dogs are cute')]

In [26]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [27]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [28]:
vector_store.add_documents(documents=documents)

['dacae53e-9067-4f5b-a8b3-d686bcae52de',
 '4e847ed6-a4ef-46a8-822e-46ae05612337',
 '0f57bfe4-25f2-4e0a-b5e2-5ae066fc7c96',
 '4181b72e-1d97-4cdf-81db-54c87d9aeb38',
 '7ee24661-2726-44c9-baf3-a561aaa306a1',
 '1c4f6237-08bb-420a-b300-69527136300d',
 'c554cf0a-0f72-4f23-8736-0e472d5a2ca8',
 '4143625a-0d57-4e81-9c41-e0aff1976274',
 '7ab86513-8f8c-4aaf-b22d-8b1568a6abd9',
 '7a7ebbcf-c3cd-409b-9d8e-78b7b229c5b3']

In [29]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(id='0f57bfe4-25f2-4e0a-b5e2-5ae066fc7c96', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4143625a-0d57-4e81-9c41-e0aff1976274', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [30]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
    
)

[Document(id='0f57bfe4-25f2-4e0a-b5e2-5ae066fc7c96', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4143625a-0d57-4e81-9c41-e0aff1976274', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='7a7ebbcf-c3cd-409b-9d8e-78b7b229c5b3', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='dacae53e-9067-4f5b-a8b3-d686bcae52de', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [31]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [32]:
result[0].metadata

{'source': 'news'}

In [33]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [34]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [35]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='0f57bfe4-25f2-4e0a-b5e2-5ae066fc7c96', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4143625a-0d57-4e81-9c41-e0aff1976274', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='7a7ebbcf-c3cd-409b-9d8e-78b7b229c5b3', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

In [37]:
vector_store.save_local("created faiss index")

In [38]:
new_vector_store=FAISS.load_local(
  "created faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [39]:
new_vector_store.similarity_search("langchain")

[Document(id='0f57bfe4-25f2-4e0a-b5e2-5ae066fc7c96', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4143625a-0d57-4e81-9c41-e0aff1976274', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='7ee24661-2726-44c9-baf3-a561aaa306a1', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='c554cf0a-0f72-4f23-8736-0e472d5a2ca8', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

If we need to invoke the ondisk index we need to create retreiver and then we can invoke

In [40]:
from langchain_community.document_loaders import PyPDFLoader

In [41]:
FILE_PATH=r"C:\Users\AGENTIC_AI\agenticai\Langchain\Vector_Databases\llama2.pdf"

In [42]:
loader=PyPDFLoader(FILE_PATH)

In [43]:
len(loader.load())

77

In [44]:
pages=loader.load()

In [45]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [46]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [47]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [48]:
split_docs = splitter.split_documents(pages)

In [49]:
len(split_docs)

615

In [50]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [51]:
vector_store.add_documents(documents=split_docs)

['bad0494d-caa3-4fb5-9c0b-bc0b62bd310a',
 'f1b7d29e-039a-4542-b2c6-57f5a9b5e670',
 '91b26f07-4569-4824-a966-96b3947249eb',
 'c21f5482-8b1b-44a7-b47c-b0fc2db29cab',
 '613c667d-9de2-4971-bbc6-8f7315969bd6',
 '879a8f0c-7366-4960-b9e1-224c8aaafaef',
 '83fd35ac-2023-47cb-95eb-0bf680315145',
 'ae356f9f-dc8d-48e6-96eb-a19d7e275ab5',
 '24ccdb35-dfb0-4fe2-982a-7a7defd55d26',
 '7ee30c47-db9a-4a47-be64-597e3d7ec178',
 '6f1b96d4-913e-4fa5-a555-0c8b9dfd13e6',
 'fad8f7f1-3a65-45f5-8b12-bf6f691f1ec5',
 'b0e5588b-48b3-4ade-bd24-fc65f312639c',
 '8cb402bf-22fe-4b60-be63-45a4bba8545e',
 '3cafd96b-d37e-442f-85be-6866c4f54967',
 '4f1311d5-b41b-4b87-a76f-d4b2101beb55',
 'b98a3b21-18b2-476d-8055-2d135ca4d849',
 '1a6bcf3c-fbf6-431d-8bdd-923b9305dab0',
 '5cc78611-e17c-489a-a4e6-6d8faf8d99b7',
 'ac5dbfaa-8cc9-4c41-8a5e-1f56510d17db',
 '6fd3b3f3-b52b-4cc1-b372-9cc8da61500e',
 '4de205eb-5281-4f86-ad35-8f20539837b9',
 'cab717fd-d749-4c8e-b40e-f0066de72783',
 '3430fcc0-7926-457d-83b0-15e26d2517ec',
 'f8638178-e225-

In [52]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [53]:
retriever.invoke("what is llama model?")

[Document(id='f8638178-e225-4bce-89b4-f3c671a12c23', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\AGENTIC_AI\\agenticai\\Langchain\\Vector_Databases\\llama2.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='0d8638d1-40fe-48e6-9ca5-2c3970942186', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00

In [55]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [56]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

##Template from hub which i am using is :

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [57]:
import pprint


In [58]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [59]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [60]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [61]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [62]:
rag_chain.invoke("what is llama model?")

"Llama is a large language model developed by Meta.  It has various versions, including tuned models for chat and pretrained models adaptable to different natural language tasks.  The model's architecture and training details are available in Meta's resources."